In [ ]:
# NERVA Market Dynamics Intelligence Engine
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

class MarketDynamicsEngine:
    def __init__(self):
        self.liquidity_data = None
        self.market_makers = {}
        self.systemic_risk_indicators = {}
        
        # Professional styling
        self.colors = {
            'high_liquidity': '#2E8B57',
            'medium_liquidity': '#FFD700', 
            'low_liquidity': '#DC143C',
            'buy_flow': '#4169E1',
            'sell_flow': '#FF6347',
            'neutral': '#708090'
        }
    
    def simulate_market_microstructure(self, n_trades=10000):
        """Simulate high-frequency market microstructure data"""
        np.random.seed(42)
        
        # Generate realistic trading data
        timestamps = pd.date_range('2024-01-01', periods=n_trades, freq='1min')
        
        # Price dynamics with microstructure noise
        price_innovations = np.random.normal(0, 0.001, n_trades)
        prices = 100 + np.cumsum(price_innovations)
        
        # Bid-ask spreads (wider during low liquidity)
        base_spread = 0.01
        spread_volatility = np.abs(np.sin(np.arange(n_trades) * 0.001)) + 0.5
        spreads = base_spread * spread_volatility + np.random.exponential(0.005, n_trades)
        
        # Order flow
        trade_sizes = np.random.exponential(1000, n_trades)
        trade_directions = np.random.choice([-1, 1], n_trades, p=[0.48, 0.52])  # Slight buy bias
        
        # Market maker presence
        mm_presence = np.random.beta(2, 5, n_trades)  # Varies throughout day
        
        # Liquidity depth
        depth_levels = np.random.gamma(2, 1000, n_trades)
        
        market_data = pd.DataFrame({
            'timestamp': timestamps,
            'price': prices,
            'bid': prices - spreads/2,
            'ask': prices + spreads/2,
            'spread': spreads,
            'trade_size': trade_sizes,
            'trade_direction': trade_directions,
            'mm_presence': mm_presence,
            'liquidity_depth': depth_levels
        })
        
        # Calculate derived metrics
        market_data['spread_bps'] = (market_data['spread'] / market_data['price']) * 10000
        market_data['order_flow'] = market_data['trade_size'] * market_data['trade_direction']
        market_data['volume'] = market_data['trade_size']
        
        print(f"Market microstructure data generated: {len(market_data)} trades")
        print(f"Average spread: {market_data['spread_bps'].mean():.2f} bps")
        print(f"Liquidity depth range: {market_data['liquidity_depth'].min():.0f} - {market_data['liquidity_depth'].max():.0f}")
        
        self.liquidity_data = market_data
        return market_data
    
    def analyze_liquidity_flows(self, window_minutes=60):
        """Analyze liquidity flow patterns"""
        if self.liquidity_data is None:
            return None
            
        data = self.liquidity_data.copy()
        data.set_index('timestamp', inplace=True)
        
        # Aggregate by time windows
        agg_data = data.resample(f'{window_minutes}min').agg({
            'order_flow': 'sum',
            'volume': 'sum',
            'spread_bps': 'mean',
            'liquidity_depth': 'mean',
            'mm_presence': 'mean',
            'price': 'last'
        })
        
        # Liquidity flow classification
        flow_threshold = agg_data['order_flow'].std()
        agg_data['flow_regime'] = pd.cut(
            agg_data['order_flow'],
            bins=[-np.inf, -flow_threshold, flow_threshold, np.inf],
            labels=['Sell Pressure', 'Balanced', 'Buy Pressure']
        )
        
        # Liquidity stress indicator
        agg_data['liquidity_stress'] = (
            (agg_data['spread_bps'] - agg_data['spread_bps'].mean()) / agg_data['spread_bps'].std() +
            (agg_data['liquidity_depth'].mean() - agg_data['liquidity_depth']) / agg_data['liquidity_depth'].std()
        ) / 2
        
        print("Liquidity flow analysis complete")
        print(f"Flow regimes distribution:")
        print(agg_data['flow_regime'].value_counts())
        
        return agg_data
    
    def detect_market_maker_behavior(self):
        """Detect and analyze market maker behavior patterns"""
        if self.liquidity_data is None:
            return None
            
        data = self.liquidity_data.copy()
        
        # Market maker activity indicators
        data['mm_activity'] = data['mm_presence'] * data['liquidity_depth']
        data['spread_contribution'] = data['mm_presence'] * (1 / data['spread_bps'])
        
        # Clustering market maker behavior
        features = data[['mm_presence', 'liquidity_depth', 'spread_bps']].dropna()
        
        if len(features) > 100:
            kmeans = KMeans(n_clusters=3, random_state=42)
            mm_clusters = kmeans.fit_predict(features)
            
            cluster_analysis = pd.DataFrame({
                'cluster': mm_clusters,
                'mm_presence': features['mm_presence'],
                'liquidity_depth': features['liquidity_depth'],
                'spread_bps': features['spread_bps']
            })
            
            cluster_summary = cluster_analysis.groupby('cluster').agg({
                'mm_presence': 'mean',
                'liquidity_depth': 'mean',
                'spread_bps': 'mean'
            })
            
            print("Market Maker Behavior Analysis:")
            print(cluster_summary)
            
            return cluster_analysis, cluster_summary
        
        return None, None
    
    def calculate_systemic_risk_indicators(self):
        """Calculate systemic risk indicators"""
        if self.liquidity_data is None:
            return None
            
        data = self.liquidity_data.copy()
        
        # 1. Liquidity Risk - spread volatility
        liquidity_risk = data['spread_bps'].rolling(100).std().mean()
        
        # 2. Market Impact - price sensitivity to order flow
        price_changes = data['price'].pct_change()
        order_flow_impact = np.corrcoef(price_changes.dropna(), data['order_flow'][1:len(price_changes)])[0,1]
        
        # 3. Market Maker Concentration
        mm_concentration = 1 - (data['mm_presence'].std() / data['mm_presence'].mean())
        
        # 4. Systemic Stress Index
        stress_components = [
            (data['spread_bps'] - data['spread_bps'].mean()) / data['spread_bps'].std(),
            (data['liquidity_depth'].mean() - data['liquidity_depth']) / data['liquidity_depth'].std(),
            (data['volume'] - data['volume'].mean()) / data['volume'].std()
        ]
        
        systemic_stress = np.mean(stress_components, axis=0)
        stress_index = np.sqrt(np.mean(systemic_stress**2))
        
        risk_indicators = {
            'liquidity_risk': liquidity_risk,
            'market_impact': abs(order_flow_impact),
            'mm_concentration': mm_concentration,
            'systemic_stress_index': stress_index
        }
        
        print("Systemic Risk Indicators:")
        for indicator, value in risk_indicators.items():
            print(f"  {indicator}: {value:.4f}")
        
        self.systemic_risk_indicators = risk_indicators
        return risk_indicators
    
    def create_market_dynamics_dashboard(self):
        """Create comprehensive market dynamics visualization"""
        
        if self.liquidity_data is None:
            return None
            
        # Get liquidity flows
        liquidity_flows = self.analyze_liquidity_flows()
        mm_clusters, mm_summary = self.detect_market_maker_behavior()
        risk_indicators = self.calculate_systemic_risk_indicators()
        
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=[
                'Price and Liquidity Depth', 'Order Flow Dynamics',
                'Bid-Ask Spread Evolution', 'Market Maker Activity',
                'Liquidity Stress Index', 'Systemic Risk Dashboard'
            ],
            specs=[[{}, {}],
                   [{}, {}],
                   [{}, {}]]
        )
        
        data = self.liquidity_data
        
        # 1. Price and liquidity depth
        fig.add_trace(
            go.Scatter(
                x=data['timestamp'],
                y=data['price'],
                mode='lines',
                name='Price',
                yaxis='y',
                line=dict(color=self.colors['neutral'])
            ), row=1, col=1
        )
        
        # Liquidity depth on secondary y-axis
        fig.add_trace(
            go.Scatter(
                x=data['timestamp'],
                y=data['liquidity_depth'],
                mode='lines',
                name='Liquidity Depth',
                yaxis='y2',
                line=dict(color=self.colors['high_liquidity'])
            ), row=1, col=1
        )
        
        # 2. Order flow
        if liquidity_flows is not None:
            colors = [self.colors['sell_flow'] if x < 0 else self.colors['buy_flow'] 
                     for x in liquidity_flows['order_flow']]
            
            fig.add_trace(
                go.Bar(
                    x=liquidity_flows.index,
                    y=liquidity_flows['order_flow'],
                    name='Order Flow',
                    marker_color=colors
                ), row=1, col=2
            )
        
        # 3. Spread evolution
        fig.add_trace(
            go.Scatter(
                x=data['timestamp'],
                y=data['spread_bps'],
                mode='lines',
                name='Bid-Ask Spread (bps)',
                line=dict(color=self.colors['medium_liquidity'])
            ), row=2, col=1
        )
        
        # 4. Market maker presence
        fig.add_trace(
            go.Scatter(
                x=data['timestamp'],
                y=data['mm_presence'],
                mode='lines',
                name='MM Presence',
                line=dict(color=self.colors['buy_flow'])
            ), row=2, col=2
        )
        
        # 5. Liquidity stress
        if liquidity_flows is not None:
            fig.add_trace(
                go.Scatter(
                    x=liquidity_flows.index,
                    y=liquidity_flows['liquidity_stress'],
                    mode='lines',
                    name='Liquidity Stress',
                    line=dict(color=self.colors['low_liquidity'])
                ), row=3, col=1
            )
            
            # Add stress threshold
            fig.add_hline(y=1, line_dash="dash", line_color="red", row=3, col=1)
        
        # 6. Risk indicators bar chart
        if risk_indicators:
            indicators = list(risk_indicators.keys())
            values = list(risk_indicators.values())
            
            fig.add_trace(
                go.Bar(
                    x=indicators,
                    y=values,
                    name='Risk Indicators',
                    marker_color=[self.colors['low_liquidity'], self.colors['medium_liquidity'],
                                self.colors['high_liquidity'], self.colors['sell_flow']]
                ), row=3, col=2
            )
        
        fig.update_layout(
            title_text="NERVA Market Dynamics Intelligence Dashboard",
            height=1200,
            showlegend=False
        )
        
        return fig

# Initialize Market Dynamics Engine
market_engine = MarketDynamicsEngine()

print("NERVA Market Dynamics Intelligence Engine initialized")
print("High-frequency market analysis protocols active")

In [ ]:
# Execute comprehensive market dynamics analysis
print("Initiating market microstructure simulation...")

# Generate market data
market_data = market_engine.simulate_market_microstructure(n_trades=5000)

print("Analyzing market dynamics...")

# Perform comprehensive analysis
liquidity_analysis = market_engine.analyze_liquidity_flows(window_minutes=30)
mm_behavior = market_engine.detect_market_maker_behavior()
risk_assessment = market_engine.calculate_systemic_risk_indicators()

# Generate intelligence dashboard
print("Creating market dynamics dashboard...")
dynamics_dashboard = market_engine.create_market_dynamics_dashboard()
if dynamics_dashboard:
    dynamics_dashboard.show()

print("Market dynamics intelligence analysis complete")
print("Systemic risk monitoring active")

# NERVA Market Dynamics Intelligence

**Classification:** ALPHA - Authorized Personnel Only  
**System:** National Economic & Risk Visual Analytics  
**Module:** Market Microstructure & Liquidity Analysis  
**Clearance Level:** RESTRICTED

---

## Intelligence Objectives:
- High-frequency market data analysis
- Liquidity flow mapping and prediction
- Market maker behavior modeling
- Systematic risk detection protocols
- Real-time market sentiment analysis

---